# Depresjon - basic feature engineering

This notebook aims to recreate feature engineering for Depresjon dataset from paper "Comparison of Night, Day and 24 h Motor Activity Data for the Classification of Depressive Episodes".

## Libraries

In [ ]:
import os
from typing import Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp

## Data loading

First, we have to load the data.

In [ ]:
data_dir = "./depresjon_data"
condition_dir = os.path.join(data_dir, "condition")
control_dir = os.path.join(data_dir, "control")
scores_file = os.path.join(data_dir, "scores.csv")

`condition` and `control` directories contain CSV files with measurements, one file per person. For example, `condition_1.csv` contains measurements for patient 1 diagnosed with depression.

Those files are read into a list of dataframes, since this makes them easy to process later.

In [ ]:
conditions = [pd.read_csv(os.path.join(condition_dir, filename)) for filename in os.listdir(condition_dir)]
controls = [pd.read_csv(os.path.join(control_dir, filename)) for filename in os.listdir(control_dir)]

In [ ]:
conditions[0].head()

The `scores.csv` file contains static information about patients.

In [ ]:
static_data = pd.read_csv(scores_file)
static_data.head()

## Exploratory data analysis

We have the data loaded, now we can explore it: check number of measurements, number of columns, their types, missing values etc. First the data for time series will be checked, then for the static data.

### Number of patients

In [ ]:
xs = ["Condition", "Control"]
ys = [len(conditions), len(controls)]

print(f"Condition number: {len(conditions)}")
print(f"Control number: {len(controls)}")

plt.bar(xs, ys)
plt.xlabel("Group")
plt.ylabel("Patients")
plt.title("Number of patients per group")
plt.show()

We definitely have quite heavy imbalance, the control group being about 50% larger than the condition group. This influences many metrics and should be taken into consideration.

### Measurements number

In [ ]:
condition_rows = pd.Series([len(df) for df in conditions])
condition_rows.describe()

In [ ]:
control_rows = pd.Series([len(df) for df in controls])
control_rows.describe()

We have greatly varying number of measurements for both condition and control group. In general measurements for control group are longer, but also more varied (both mean and standard deviation are higher). For fair assessment, where each patient gets the same chance to influence the model, the measurements should be of equal length for all cases. However, for calculating aggregations (e.g. mean / median value) this may not be necessary, since they typically reduce the data to a single number.

### Columns and types

In [ ]:
conditions[0].head()

In [ ]:
conditions[0].dtypes

For measurement files we have 3 columns:
- timestamp with precise measurement date and time, with 1 minute measurement resolution
- date, information already included in the timestamp
- activity, the time series value

Data types of columns are definitely wrong - `timestamp` should be a proper timestamp and the `date` column is redundant. This will be fixed in the preprocessing section.

### Missing values

In [ ]:
condition_NaNs = pd.Series([df["activity"].isna().sum() for df in conditions])
condition_NaNs.mean()

In [ ]:
control_NaNs = pd.Series([df["activity"].isna().sum() for df in controls])
control_NaNs.mean()

We have exactly 0 missing values for the dependent variable, activity. This means that the data was accurately gathered for all patients for the entire duration of measurement period.

### Statis data

In [ ]:
static_data.head()

In [ ]:
static_data.dtypes

In [ ]:
for col in static_data.columns:
    print(col, static_data[col].unique())

Observations:
- all data types seem correct
- `number` is essentially an index for patients and should not be used as a feature
- `days` indicate the number of data collection days, but this information is already included in the time series timestamps
- `gender` is correct, but doesn't follow the usual convention of 0-1 values for binary features
- `age` should be preprocessed to integers (e.g. `0` for <50 years, `1` for >= 50 years) for classification
- `afftype` and `melanch` indicate the clinical state observations for depressed patients and are NaN for non-depressed controls
- `edu`, `marriage` and `work` explain socioeconomic status of the patient
- `madrs1` and `madrs2` are MADRS score for patients with condition at the beginning and at the end of measurements; they are not used for classification, but they could be used as regression targets

## Feature engineering

In the paper several steps of feature engineering are introduced. They need to be performed, as the typical models for tabular data like Random Forest are used. This approach allows usage of classical ML algorithms on time series data, while also indirectly incorporating time dependencies in form of features derived from the signal.

### Preprocessing

**Warning**: I've found out that the paper is actually inconsistent in what it says and does in the preprocessing. It states that:
"For the pre-processing stage, the next step are proposed. Since the total amount of data recorded for each subject is different, a new subset of data is extracted, adjusting the number of observations to be equal for each subject. Theh, from the new set of data, a segmentation is applied to form one hour data intervals. This segmentation allowed the classification of depressive episodes per hour.

Therefore, based on the hourly segmentation, three different subsets are constructed; night motor activity (from 21 to 7 h taking into account the sunrise standard hours) [21], day motor activity (from 8 to 20 h) and finally all day motor activity with the total day hours. The number of observations contained in each dataset is shown in Table 1. After separated the data into day, night and 24 h data were cleaned from missing data."

According to this, we should:
- trim observations to the length of the shortest one, forming "new set of data" with same number of observations per subject
- segment into 1-hour intervals, calculating average activity
- create night dataset, day dataset and dataset with all observations

According to the paper, they got the following number of hourly observations:
- day: 14168
- night: 11945
- full data: 26113

But this number is wrong. This is approximately the number of raw observations in the dataset, not number of hour segments. In addition, to make sure that night and day data has the same length, it should be trimmed to the same length after splitting. Corrected process:
- segment data for each patient into 1-hour intervals, calculating average activity
- create night dataset, day dataset and dataset with all observations
- trim observations in each dataset to the length of the shortest one

This way, we arrive at the following sequences lengths:
- day: 9845
- night: 7865
- full data: 17710

First, correct the data type of `timestamp` column and drop the redundant `date` column.

In [ ]:
for df in (conditions + controls):
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d %H:%M:%S")

for df in (conditions + controls):
    if "date" in df.columns:
        df.drop("date", axis=1, inplace=True)

conditions[0].head()

In [ ]:
controls[1].dtypes

Next, group by hour and calculate mean hourly activity.

In [ ]:
def group_by_hour(df: pd.DataFrame) -> pd.DataFrame:
    # group by hour, calculate means
    timestamps = df["timestamp"]
    group_by_cols = [timestamps.dt.year, timestamps.dt.month, timestamps.dt.day, timestamps.dt.hour]
    grouped = df.groupby(group_by_cols).mean()

    # recreate flat index for 2D table
    grouped.index = grouped.index.to_flat_index()
    datetime_index_df = pd.DataFrame(grouped.index.values.tolist(), columns=["year", "month", "day", "hour"])
    datetime_index = pd.to_datetime(datetime_index_df)

    # add back the datetime information
    grouped.reset_index(inplace=True)
    grouped["datetime"] = datetime_index
    grouped.drop("index", axis=1, inplace=True)

    # change column order for readability
    grouped = grouped.reindex(["datetime", "activity"], axis=1)
    
    return grouped

In [ ]:
conditions_preprocessed = [group_by_hour(df) for df in conditions]
controls_preprocessed = [group_by_hour(df) for df in controls]

Lastly, create separate night and day datasets in addition to the whole dataset.

In [ ]:
def get_night_day_division(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    night_df = df.loc[(df["datetime"].dt.hour >= 21) | (df["datetime"].dt.hour < 8)]
    day_df = df.loc[(df["datetime"].dt.hour >= 8) & (df["datetime"].dt.hour < 21)]
    return night_df, day_df

In [ ]:
conditions_night = []
conditions_day = []

controls_night = []
controls_day = []

for df in conditions_preprocessed:
    night_df, day_df = get_night_day_division(df)
    conditions_night.append(night_df)
    conditions_day.append(day_df)

for df in controls_preprocessed:
    night_df, day_df = get_night_day_division(df)
    controls_night.append(night_df)
    controls_day.append(day_df)

Trim to the same number of 1-hour segments, so we have equal length sequences for night, day and all data.

In [ ]:
min_length_night = min([len(df) for df in conditions_night + controls_night])
min_length_day = min([len(df) for df in conditions_day + controls_day])
min_length_all = min(min_length_night, min_length_day)

conditions_trimmed = {
    "night": [df[:min_length_night] for df in conditions_night],
    "day": [df[:min_length_day] for df in conditions_day],
    "all": [df[:min_length_all] for df in conditions_night + conditions_day]
}

controls_trimmed = {
    "night": [df[:min_length_night] for df in controls_night],
    "day": [df[:min_length_day] for df in controls_day],
    "all": [df[:min_length_all] for df in controls_night + controls_day]
}

Let's check the number of hour segments.

In [ ]:
night_observations = sum([len(df) for df in conditions_trimmed["night"] + controls_trimmed["night"]])
day_observations = sum([len(df) for df in conditions_trimmed["day"] + controls_trimmed["day"]])
all_observations= sum([len(df) for df in conditions_trimmed["all"] + controls_trimmed["all"]])

print(f"Day: {day_observations}")
print(f"Night: {night_observations}")
print(f"Full data: {day_observations + night_observations}")

As you can see, the number of segments is very different than stated in the paper. However, that was probably just a mistake in specifying the table contents and authors got similiar numbers of actual hourly segments as here.

Before further processing it will come in handy to work on regular Numpy arrays instead of lists of DataFrames. We will have 1 row per patient, with columns indicating measurements (short and wide matrix).

In [ ]:
X_night = [df["activity"].values for df in conditions_trimmed["night"]] + \
          [df["activity"].values for df in controls_trimmed["night"]]
X_night = np.vstack(X_night)

y_night = np.zeros(X_night.shape[0])
y_night[:len(conditions_trimmed["night"]) + 1] = 1


X_day = [df["activity"].values for df in conditions_trimmed["day"]] + \
        [df["activity"].values for df in controls_trimmed["day"]]
X_day = np.vstack(X_day)

y_day = np.zeros(X_day.shape[0])
y_day[:len(conditions_trimmed["day"]) + 1] = 1


X_all = [df["activity"].values for df in conditions_trimmed["all"]] + \
        [df["activity"].values for df in controls_trimmed["all"]]
X_all = np.vstack(X_all)

y_all = np.zeros(X_all.shape[0])
y_all[:len(conditions_trimmed["all"]) + 1] = 1

Make sure that the data is correct:

In [ ]:
X_night[:5]

In [ ]:
y_night

There seem to be no missing values, despite what's described in the paper. There is a possibility that dataset available online has already been cleaned in this regard.

Next we standardize the data, i.e. subtract mean and divide by standard deviation. I assume that standardization is done separately for all 3 sets of data, though this has not been specified in the paper.

In [ ]:
X_night_standardized = (X_night - X_night.mean()) / X_night.std()
X_day_standardized = (X_day - X_day.mean()) / X_day.std()
X_all_standardized = (X_all - X_all.mean()) / X_all.std()

### Time domain

Time features are extracted according to the article:
- `mean`, `median`, `stddev`, `variance`, `kurtosis`, `minimum`, `maximum` - quite self explanatory statistical features
- `coeff_of_var` - coefficient of variation, the ratio of the biased standard deviation to the mean
- `iqr` - interquartile range, difference between 75 and 25 percentile (3rd and 1st quartile)
- `trimmed_mean` - alternatively truncated mean, mean of the values where the most extreme values (from both ends) are not used; since the article doesn't specify this, I assume that the popular 10% trim percentage is used

Data is saved as a DataFrame, since some machine learning models can provide additional insight when using named columns.

In [ ]:
from scipy.stats import iqr, kurtosis, trim_mean, variation

In [ ]:
def extract_features(X: np.ndarray) -> pd.DataFrame:
    features = {
        "mean": X.mean(axis=1),
        "median": np.median(X, axis=1),
        "stddev": X.std(axis=1),
        "variance": np.var(X, axis=1),
        "kurtosis": kurtosis(X, axis=1),
        "coeff_of_var": variation(X, axis=1),
        "iqr": iqr(X, axis=1),
        "minimum": X.min(axis=1),
        "maximum": X.max(axis=1),
        "trimmed_mean": trim_mean(X, proportiontocut=0.1, axis=1)
    }
    return pd.DataFrame(features)

In [ ]:
X_night_features = extract_features(X_night_standardized)
X_day_features = extract_features(X_day_standardized)
X_all_features = extract_features(X_all_standardized)

### Frequency domain

#### First approach

In [ ]:
f_frames = []

for c_i, c in enumerate(conditions, start=1):
    c_copy = c.copy()
    c_copy['number'] = f'condition_{c_i}'
    f_frames.append(c_copy)
    
for c_i, c in enumerate(controls, start=1):
    c_copy = c.copy()
    c_copy['number'] = f'control_{c_i}'
    f_frames.append(c_copy)

f_df = pd.concat(f_frames)

f_df['date'] = f_df.apply(lambda r: r['timestamp'].date(), axis=1)
f_df['hour'] = f_df.apply(lambda r: r['timestamp'].hour, axis=1)

f_df.reset_index()
f_df.head()

In [ ]:
f_df.drop('timestamp', axis=1, inplace=True)
f_df.head()

In [ ]:
f_df_segments = f_df.groupby(['number', 'date', 'hour'])
f_df_segments

In [ ]:
f_df_grouped = f_df_segments['activity'].apply(list).to_frame()
f_df_grouped

In [ ]:
from scipy.fft import fft

# Now:
# 1. Adjust the number of samples for each 'number'
# 2. Create three subsets: night, day, all
# 3. Standardize
# 4. Do the feature extraction

In [ ]:
# 4. Frequency-domain feature extraction
f_df_grouped['f_activity'] = f_df_grouped.apply(lambda r: fft(r['activity']), axis=1)
f_df_grouped

#### Second approach: hourly_mean  > FFT > PSD

In [ ]:
from scipy.fft import fft
from scipy.signal import welch, periodogram
from scipy.stats import entropy, moment

def as_psd(X: np.ndarray, fs=1):
    frequencies, psd_estimates = periodogram(X, fs, axis=-1)
    return frequencies, psd_estimates

def extract_frequency_features(X: np.ndarray, fs=1) -> pd.DataFrame:
    X_fft = fft(X, axis=1)
    X_psd_freqs, X_psd = as_psd(X, fs)
    X_psd_avgs = np.sum(np.power(np.absolute(X_fft), 2), axis=1) / X_psd_freqs[-1]
    
    def spectral_flatness(ns):
        norm = ns.mean()
        norm = 1 if norm == 0 else norm
        return np.exp(np.log(ns + 1e-20).mean()) / norm
    
    features_df = extract_features(X_psd)
    features_df['spectral_density'] = X_psd_avgs
    features_df['entropy'] = entropy(X_psd, base=2, axis=1) 
    features_df['skewness'] = moment(X_psd, moment=3, axis=1)
    features_df['spectral_flatness'] = np.apply_along_axis(spectral_flatness, arr=X_psd, axis=1)
    
    features_df = features_df.add_prefix('freq_')
    return features_df

get_freq_features = lambda X: extract_frequency_features(X, fs=1./3600) # 1 hour sampling frequency

X_day_freq_features = get_freq_features(X_day_standardized)
X_night_freq_features = get_freq_features(X_night_standardized)
X_all_freq_features = get_freq_features(X_all_standardized)

In [ ]:
X_all_freq_features

### Feature selection